**Copyright 2021 The TF-Agents Authors.**

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Actor-Learner API를 사용한 SAC minitaur

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/7_SAC_minitaur_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>
</td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/7_SAC_minitaur_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/agents/blob/master/docs/tutorials/7_SAC_minitaur_tutorial.ipynb"><img>GitHub에서 소그 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/agents/docs/tutorials/7_SAC_minitaur_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>


## 소개

이 예는 [Minitaur](https://arxiv.org/abs/1812.05905) 환경에서 [Soft Actor Critic](https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/bullet/minitaur.py) 에이전트를 훈련하는 방법을 보여줍니다.

[DQN Colab](https://github.com/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb)을 통해 작업했다면 이 내용이 매우 친숙할 것입니다. 주목할만한 변경 사항은 다음과 같습니다.

- 에이전트를 DQN에서 SAC로 변경합니다.
- CartPole보다 훨씬 복잡한 환경인 Minitaur에서 훈련합니다. Minitaur 환경은 네 발 달린 로봇이 전진하도록 훈련하는 데 목표를 두고 있습니다.
- 분산 강화 학습을 위해 TF-Agents Actor-Learner API를 사용합니다.

이 API는 경험 재현 버퍼와 변수 컨테이너(매개변수 서버)를 사용하는 분산 데이터 수집과 여러 기기에 걸친 분산 훈련을 모두 지원합니다. 이 API는 매우 간단하고 모듈식으로 설계되었습니다. [Reverb](https://deepmind.com/research/open-source/Reverb)는 재현 버퍼 및 가변 컨테이너 모두에 사용하고 [TF DistributionStrategy API](https://www.tensorflow.org/guide/distributed_training)는 GPU 및 TPU에서의 분산 훈련에 사용합니다.

다음과 같은 종속성을 설치하지 않은 경우, 다음을 실행합니다.

In [ ]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg
!pip install 'imageio==2.4.0'
!pip install matplotlib
!pip install tf-agents[reverb]
!pip install pybullet

## 설정

먼저 필요한 여러 도구를 가져옵니다.

In [ ]:
import base64
import imageio
import IPython
import matplotlib.pyplot as plt
import os
import reverb
import tempfile
import PIL.Image

import tensorflow as tf

from tf_agents.agents.ddpg import critic_network
from tf_agents.agents.sac import sac_agent
from tf_agents.agents.sac import tanh_normal_projection_network
from tf_agents.environments import suite_pybullet
from tf_agents.metrics import py_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import greedy_policy
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_py_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.train import actor
from tf_agents.train import learner
from tf_agents.train import triggers
from tf_agents.train.utils import spec_utils
from tf_agents.train.utils import strategy_utils
from tf_agents.train.utils import train_utils

tempdir = tempfile.gettempdir()

## 하이퍼 매개변수

In [ ]:
env_name = "MinitaurBulletEnv-v0" # @param {type:"string"}

# Use "num_iterations = 1e6" for better results (2 hrs)
# 1e5 is just so this doesn't take too long (1 hr)
num_iterations = 100000 # @param {type:"integer"}

initial_collect_steps = 10000 # @param {type:"integer"}
collect_steps_per_iteration = 1 # @param {type:"integer"}
replay_buffer_capacity = 10000 # @param {type:"integer"}

batch_size = 256 # @param {type:"integer"}

critic_learning_rate = 3e-4 # @param {type:"number"}
actor_learning_rate = 3e-4 # @param {type:"number"}
alpha_learning_rate = 3e-4 # @param {type:"number"}
target_update_tau = 0.005 # @param {type:"number"}
target_update_period = 1 # @param {type:"number"}
gamma = 0.99 # @param {type:"number"}
reward_scale_factor = 1.0 # @param {type:"number"}

actor_fc_layer_params = (256, 256)
critic_joint_fc_layer_params = (256, 256)

log_interval = 5000 # @param {type:"integer"}

num_eval_episodes = 20 # @param {type:"integer"}
eval_interval = 10000 # @param {type:"integer"}

policy_save_interval = 5000 # @param {type:"integer"}

## 환경

RL의 환경은 우리가 해결하려고 하는 작업 또는 문제를 나타냅니다. `suites`를 사용하여 TF-Agents에서 표준 환경을 쉽게 만들 수 있습니다. 문자열 환경 이름을 고려하여 OpenAI Gym, Atari, DM Control 등과 같은 소스에서 환경을 로드하기 위한 여러 `suites`가 마련되어 있습니다.

이제 Pybullet Suite에서 Minituar 환경을 로드하겠습니다.

In [ ]:
env = suite_pybullet.load(env_name)
env.reset()
PIL.Image.fromarray(env.render())

이 환경의 목표는 에이전트가 Minitaur 로봇을 제어하고 가능한 빨리 전진하도록 하는 정책을 훈련하는 것입니다. 에피소드는 1000개 스텝 동안 유지되며 이익은 에피소드 전체에서 보상의 합계입니다.

정책이 `actions`를 생성하는 데 사용하는 `observation`로 환경이 제공하는 정보를 살펴보겠습니다.

In [ ]:
print('Observation Spec:')
print(env.time_step_spec().observation)
print('Action Spec:')
print(env.action_spec())

보이는 바와 같이 관찰이 상당히 복잡합니다. 모든 모터의 각도, 속도 및 토크를 나타내는 28개의 값을 받습니다. 그러면 환경이 `[-1, 1]` 사이의 행동에 대해 8개의 값을 예상합니다. 이들 값이 기대하는 모터 각도입니다.

일반적으로, 훈련 중 데이터 수집을 위한 환경과 평가를 위한 환경의 두 가지 환경을 만듭니다. 이들 환경은 순수 Python으로 작성되고 Actor Learner API가 직접 사용하는 numpy 배열을 사용합니다.

In [ ]:
collect_env = suite_pybullet.load(env_name)
eval_env = suite_pybullet.load(env_name)

## 배포 전략

DistributionStrategy API를 사용하여 데이터 병렬 처리를 사용하는 다중 GPU 또는 TPU와 같은 여러 기기에서 훈련 스텝 계산을 실행할 수 있습니다. 훈련 스텝은 다음과 같습니다.

- 훈련 데이터의 배치를 받습니다.
- 여러 기기에 이 데이터를 분할합니다.
- 정방향 스텝을 계산합니다.
- 손실의 MEAN을 집계하고 계산합니다.
- 역방향 스텝을 계산하고 그래디언트 변수 업데이트를 수행합니다.

TF-Agents Learner API 및 DistributionStrategy API를 사용하면 아래의 훈련 로직을 변경하지 않고도 GPU의 훈련 스텝 실행(MirroredStrategy 사용)을 TPU(TPUStrategy 사용)로 전환하기가 매우 쉽습니다.

### GPU 활성화

GPU에서 실행하려면 먼저 노트북에서 사용할 수 있게 GPU를 활성화해야합니다.

- Edit→Notebook Settings로 이동합니다.
- Hardware Accelerator 드롭다운에서 GPU를 선택합니다.

### 전략 선택

`strategy_utils`를 사용하여 전략을 생성합니다. 내부적으로, 다음 매개변수를 전달합니다.

- `use_gpu = False`는 CPU를 사용하는 `tf.distribute.get_strategy()`를 반환합니다.
- `use_gpu = True`는 하나의 머신에서 TensorFlow에 보이는 모든 GPU를 사용하는 `tf.distribute.MirroredStrategy()`를 반환합니다.

In [ ]:
use_gpu = True #@param {type:"boolean"}

strategy = strategy_utils.get_strategy(tpu=False, use_gpu=use_gpu)

아래에서 볼 수 있듯이 모든 변수와 에이전트는 `strategy.scope()` 아래에 생성되어야 합니다.

## 에이전트

SAC 에이전트를 만들려면 먼저 훈련할 네트워크를 만들어야 합니다. SAC는 actor-critic 에이전트이므로 두 개의 네트워크가 필요합니다.

Critic은 `Q(s,a)`에 대한 추정치를 제공합니다. 즉, 관찰값과 행동을 입력으로 받고 주어진 상태에 대해 행동이 얼마나 좋은지를 추정합니다.


In [ ]:
observation_spec, action_spec, time_step_spec = (
      spec_utils.get_tensor_specs(collect_env))

with strategy.scope():
  critic_net = critic_network.CriticNetwork(
        (observation_spec, action_spec),
        observation_fc_layer_params=None,
        action_fc_layer_params=None,
        joint_fc_layer_params=critic_joint_fc_layer_params,
        kernel_initializer='glorot_uniform',
        last_kernel_initializer='glorot_uniform')

이 critic을 사용하여 `actor` 네트워크를 훈련하면 관찰값에 따라 행동을 생성할 수 있습니다.

`ActorNetwork`는 tanh 함수 제한(tanh-squashed) [MultivariateNormalDiag](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/MultivariateNormalDiag) 분포를 위한 매개변수를 예측합니다. 그런 다음 이 분포는 행동을 생성해야 할 때마다 현재 관찰값에 따라 샘플링됩니다.

In [ ]:
with strategy.scope():
  actor_net = actor_distribution_network.ActorDistributionNetwork(
      observation_spec,
      action_spec,
      fc_layer_params=actor_fc_layer_params,
      continuous_projection_net=(
          tanh_normal_projection_network.TanhNormalProjectionNetwork))

이러한 네트워크를 통해 이제 에이전트를 인스턴스화할 수 있습니다.


In [ ]:
with strategy.scope():
  train_step = train_utils.create_train_step()

  tf_agent = sac_agent.SacAgent(
        time_step_spec,
        action_spec,
        actor_network=actor_net,
        critic_network=critic_net,
        actor_optimizer=tf.keras.optimizers.Adam(
            learning_rate=actor_learning_rate),
        critic_optimizer=tf.keras.optimizers.Adam(
            learning_rate=critic_learning_rate),
        alpha_optimizer=tf.keras.optimizers.Adam(
            learning_rate=alpha_learning_rate),
        target_update_tau=target_update_tau,
        target_update_period=target_update_period,
        td_errors_loss_fn=tf.math.squared_difference,
        gamma=gamma,
        reward_scale_factor=reward_scale_factor,
        train_step_counter=train_step)

  tf_agent.initialize()

## 재현 버퍼

환경에서 수집된 데이터를 추적하기 위해 Deepmind의 효율적이고 확장 가능하며 사용하기 쉬운 재현 시스템인 [Reverb](https://deepmind.com/research/open-source/Reverb)를 사용합니다. 이를 통해 Actor가 수집하고 Learner가 훈련 중에 소비하는 경험 데이터를 저장합니다.

이 튜토리얼에서는 `max_size`보다 덜 중요하지만 비동기 수집 및 훈련을 포함한 분산 설정에서 2~1000개 사이의 samples_per_insert를 사용하여 `rate_limiters.SampleToInsertRatio`를 실험할 수 있습니다. 예를 들면 다음과 같습니다.

```
rate_limiter=reverb.rate_limiters.SampleToInsertRatio(samples_per_insert=3.0, min_size_to_sample=3, error_buffer=3.0)
```


In [ ]:
table_name = 'uniform_table'
table = reverb.Table(
    table_name,
    max_size=replay_buffer_capacity,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1))

reverb_server = reverb.Server([table])

재현 버퍼는 저장될 텐서를 설명하는 사양을 사용하여 구성되며 `tf_agent.collect_data_spec`을 사용하여 에이전트에서 가져올 수 있습니다.

SAC Agent에는 손실 계산을 위해 현재와 다음 관찰 값이 모두 필요하기 때문에 `sequence_length=2`를 설정합니다.

In [ ]:
reverb_replay = reverb_replay_buffer.ReverbReplayBuffer(
    tf_agent.collect_data_spec,
    sequence_length=2,
    table_name=table_name,
    local_server=reverb_server)

이제 Reverb 재현 버퍼에서 TensorFlow 데이터세트를 생성합니다. Learner에 이 데이터세트를 전달하여 훈련을 위한 경험을 샘플링합니다.

In [ ]:
dataset = reverb_replay.as_dataset(
      sample_batch_size=batch_size, num_steps=2).prefetch(50)
experience_dataset_fn = lambda: dataset

## 정책

TF-Agents에서 정책은 RL의 표준 정책 개념을 나타냅니다. 즉, 주어진 `time_step`에서 행동 또는 행동에 대한 분포를 생성합니다. 기본 메서드는 `policy_step = policy.step(time_step)`이고, 여기서 `policy_step`은 명명된 튜플 `PolicyStep(action, state, info)`입니다. `policy_step.action`은 환경에 적용할 `action`이고 `state`는 상태 저장(RNN) 정책에 대한 상태를 나타내며 `info`에는 행동의 로그 확률 등의 보조 정보가 포함될 수 있습니다.

에이전트에는 두 가지 정책이 있습니다.

- `agent.policy` — 평가 및 배포에 사용되는 기본 정책입니다.
- `agent.collect_policy` — 데이터 수집에 사용되는 두 번째 정책입니다.

In [ ]:
tf_eval_policy = tf_agent.policy
eval_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_eval_policy, use_tf_function=True)

In [ ]:
tf_collect_policy = tf_agent.collect_policy
collect_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_collect_policy, use_tf_function=True)

에이전트와 독립적으로 정책을 만들 수 있습니다. 예를 들어, `tf_agents.policies.random_tf_policy`를 사용하여 각 time_step 동안 행동을 무작위로 선택하는 정책을 생성합니다.

In [ ]:
random_policy = random_py_policy.RandomPyPolicy(
  collect_env.time_step_spec(), collect_env.action_spec())

## 행위자(Actor)

행위자는 정책과 환경 간의 상호 작용을 관리합니다.

- Actor 구성 요소에는 환경 인스턴스(`py_environment`)와 정책 변수의 복사본이 포함됩니다.
- 각 Actor 작업자는 정책 변수의 로컬 값이 주어지면 일련의 데이터 수집 스텝을 실행합니다.
- `actor.run()`을 호출하기 전에 훈련 스크립트에서 변수 컨테이너 클라이언트 인스턴스를 사용하여 변수 업데이트를 명시적으로 수행합니다.
- 관찰된 경험은 각 데이터 수집 단계에서 재현 버퍼에 기록됩니다.

데이터 수집 스텝을 실행할 때 Actor는 관찰자에게 (상태, 행동, 보상)의 궤적을 전달하여 Reverb 재현 시스템에 캐싱하고 기록하도록 합니다.

`stride_length=1`이므로 프레임 [(t0,t1) (t1,t2) (t2,t3), ...]에 대한 궤적을 저장합니다.

In [ ]:
rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  reverb_replay.py_client,
  table_name,
  sequence_length=2,
  stride_length=1)

무작위 정책으로 Actor를 만들고 재현 버퍼를 시드할 경험을 수집합니다.

In [ ]:
initial_collect_actor = actor.Actor(
  collect_env,
  random_policy,
  train_step,
  steps_per_run=initial_collect_steps,
  observers=[rb_observer])
initial_collect_actor.run()

수집 정책으로 Actor를 인스턴스화하여 훈련 중에 더 많은 경험을 수집합니다.

In [ ]:
env_step_metric = py_metrics.EnvironmentSteps()
collect_actor = actor.Actor(
  collect_env,
  collect_policy,
  train_step,
  steps_per_run=1,
  metrics=actor.collect_metrics(10),
  summary_dir=os.path.join(tempdir, learner.TRAIN_DIR),
  observers=[rb_observer, env_step_metric])

훈련 중에 정책을 평가하는 데 사용할 Actor를 만듭니다. 나중에 메트릭을 기록하기 위해 `actor.eval_metrics(num_eval_episodes)`를 전달합니다.

In [ ]:
eval_actor = actor.Actor(
  eval_env,
  eval_policy,
  train_step,
  episodes_per_run=num_eval_episodes,
  metrics=actor.eval_metrics(num_eval_episodes),
  summary_dir=os.path.join(tempdir, 'eval'),
)

## 학습자(Learner)

Learner 구성 요소는 에이전트를 포함하고, 재현 버퍼의 경험 데이터를 사용하여 정책 변수에 그래디언트 스텝 업데이트를 수행합니다. 하나 이상의 훈련 스텝 후에 Learner는 새로운 변수 값 세트를 변수 컨테이너에 푸시할 수 있습니다.

In [ ]:
saved_model_dir = os.path.join(tempdir, learner.POLICY_SAVED_MODEL_DIR)

# Triggers to save the agent's policy checkpoints.
learning_triggers = [
    triggers.PolicySavedModelTrigger(
        saved_model_dir,
        tf_agent,
        train_step,
        interval=policy_save_interval),
    triggers.StepPerSecondLogTrigger(train_step, interval=1000),
]

agent_learner = learner.Learner(
  tempdir,
  train_step,
  tf_agent,
  experience_dataset_fn,
  triggers=learning_triggers,
  strategy=strategy)

## 메트릭 및 평가

위의 `actor.eval_metrics`로 eval Actor를 인스턴스화하여 정책 평가 중에 가장 일반적으로 사용되는 메트릭을 생성합니다.

- 평균 이익, 이익은 에피소드에 대한 환경에서 정책을 실행하는 동안 얻은 보상의 합계이며 일반적으로 몇 에피소드에 걸쳐 평균을 구합니다.
- 평균 에피소드 길이

Actor를 실행하여 이들 메트릭을 생성합니다.

In [ ]:
def get_eval_metrics():
  eval_actor.run()
  results = {}
  for metric in eval_actor.metrics:
    results[metric.name] = metric.result()
  return results

metrics = get_eval_metrics()

In [ ]:
def log_eval_metrics(step, metrics):
  eval_results = (', ').join(
      '{} = {:.6f}'.format(name, result) for name, result in metrics.items())
  print('step = {0}: {1}'.format(step, eval_results))

log_eval_metrics(0, metrics)

다른 메트릭의 기타 표준 구현에 대해서는 [메트릭 모듈](https://github.com/tensorflow/agents/blob/master/tf_agents/metrics/tf_metrics.py)을 확인하세요.

## 에이전트 훈련하기

훈련 루프에는 환경에서 데이터를 수집하는 것과 에이전트의 네트워크를 최적화하는 것이 포함됩니다. 그 과정에서 이따금 에이전트의 정책을 평가하여 진행 상황을 파악합니다.

In [ ]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = get_eval_metrics()["AverageReturn"]
returns = [avg_return]

for _ in range(num_iterations):
  # Training.
  collect_actor.run()
  loss_info = agent_learner.run(iterations=1)

  # Evaluating.
  step = agent_learner.train_step_numpy

  if eval_interval and step % eval_interval == 0:
    metrics = get_eval_metrics()
    log_eval_metrics(step, metrics)
    returns.append(metrics["AverageReturn"])

  if log_interval and step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, loss_info.loss.numpy()))

rb_observer.close()
reverb_server.stop()

## 시각화


### 플롯

에이전트의 성능을 확인하기 위해 평균 이익 대 글로벌 스텝을 플롯할 수 있습니다. `Minitaur`에서 보상 함수는 minitaur가 1000개 스텝에서 얼마나 멀리까지 가는지를 기준으로 하며, 에너지 소비에 불이익을 줍니다.

In [ ]:
#@test {"skip": true}

steps = range(0, num_iterations + 1, eval_interval)
plt.plot(steps, returns)
plt.ylabel('Average Return')
plt.xlabel('Step')
plt.ylim()

### 비디오

각 스텝에서 환경을 렌더링하여 에이전트의 성능을 시각화하면 도움이 됩니다. 이를 수행하기 전에 먼저 이 Colab에 비디오를 포함하는 함수를 작성하겠습니다.

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

다음 코드는 몇 가지 에피소드에 대한 에이전트 정책을 시각화합니다.

In [ ]:
num_episodes = 3
video_filename = 'sac_minitaur.mp4'
with imageio.get_writer(video_filename, fps=60) as video:
  for _ in range(num_episodes):
    time_step = eval_env.reset()
    video.append_data(eval_env.render())
    while not time_step.is_last():
      action_step = eval_actor.policy.action(time_step)
      time_step = eval_env.step(action_step.action)
      video.append_data(eval_env.render())

embed_mp4(video_filename)